In [547]:
aminoacids = {"A": 89.1,
              "R": 174.2,
              "N": 132.1,
              "D": 133.1,
              "C": 121.2,
              "Q": 146.1,
              "E": 147.1,
              "G": 75,
              "H": 155.2,
              "I": 131.2,
              "L": 131.2,
              "K": 146.2,
              "M": 149.2,
              "F": 165.2,
              "P": 115.1,
              "S": 105.1,
              "T": 119.1,
              "W": 204.2,
              "Y": 181.2,
              "V": 117.1,
              "O": 132.1
              }

читаем данные с файла и окгруляем их

In [548]:
with open('Spectrum_task2.txt') as f:
    spectrum = list(map(float, f.read().split()))
    spectrum = list(map(round, spectrum))
    
print(spectrum)

[372, 375, 390, 392, 409, 420, 427, 443, 446, 461, 471, 477, 491, 505, 506, 519, 536, 546, 553, 562, 588, 600, 616, 617, 618, 633, 634, 636, 652, 652, 702, 703, 712, 718, 721, 730, 749, 763, 763, 764, 780, 780, 781, 782, 797, 862, 876, 877, 879, 879, 893, 894, 895, 896, 927, 944, 976, 976, 977, 979, 1006, 1008, 1022, 1024, 1024, 1040, 1040, 1042, 1043, 1058, 1120, 1121, 1138, 1139, 1140, 1156, 1158, 1169, 1172, 1185, 1221, 1222, 1224, 1240, 1241, 1250, 1256, 1266, 1268, 1269]


создаем из аминокислот кусочки для спектра

In [549]:
peptids = dict()
for el in aminoacids:
    peptids[el] = int(aminoacids[el] - 18)
print(peptids)    

{'A': 71, 'R': 156, 'N': 114, 'D': 115, 'C': 103, 'Q': 128, 'E': 129, 'G': 57, 'H': 137, 'I': 113, 'L': 113, 'K': 128, 'M': 131, 'F': 147, 'P': 97, 'S': 87, 'T': 101, 'W': 186, 'Y': 163, 'V': 99, 'O': 114}


смотрим разницу между всеми элементами спектра, которая попадает в диапазон аминкислот,
чтобы узнать кто должен присутствовать в нашем белке

In [550]:
differ = dict()

for m in spectrum:
    for n in spectrum:
        if 190 >= round(m - n, 1) >= 57:
            if m - n not in differ:
                differ[m - n] = 1
            else:
                differ[m - n] += 1

dif_sort = sorted(differ.items(), key=lambda item: item[1])[::-1]

often = dict()

for l in dif_sort:
    for m, k in peptids.items():
        if l[0] == k:
            print(m, l[1], 'frequence', peptids[m], 'mass', end='\n')
            often[m] = l[1]
            

P 24 frequence 97 mass
F 23 frequence 147 mass
Y 22 frequence 163 mass
E 20 frequence 129 mass
V 20 frequence 99 mass
N 19 frequence 114 mass
O 19 frequence 114 mass
I 19 frequence 113 mass
L 19 frequence 113 mass
D 16 frequence 115 mass
Q 13 frequence 128 mass
K 13 frequence 128 mass
M 13 frequence 131 mass
A 9 frequence 71 mass
T 8 frequence 101 mass
C 6 frequence 103 mass
R 6 frequence 156 mass
S 5 frequence 87 mass
G 2 frequence 57 mass
H 1 frequence 137 mass


Получили 19 аминокислот, ну круто. Не очень понимаю что из этого отбрасывать.

С учётом того что в будущем придётся проверять насколько хорошо спектр предполагаемого пептида совпадает с экспериментальным, то напишу функцию для подсчёта score конкретного пептида

In [660]:
def score(peptid: str) -> tuple:
    '''
    Смотрим какое количество подстрок будет находится в спектре с учетом того, что строка циклическая
    '''
    score = 0

#     substrings = set()
#     n = len(peptid)
#     for i in range(n):
#         for j in range(i + 1, n + 1):
#             substrings.add(peptid[i:j])
            
    cyclic_string = peptid + peptid
    substrings = set()

    for i in range(len(peptid)):
        for j in range(i + 1, i + len(peptid) + 1):
            substrings.add(cyclic_string[i:j])
            
    for var in substrings:
        t = 0
        for char in var:
            t += peptids[char]
        if t in spectrum:
            score += 1
            
    return (peptid,score)

смотрим всевозможные сочетания с повторениями для поиска их в файле спектра. W не учитываем, т.к. ни разу не встретилась

In [661]:
from itertools import *

c3_20 = dict()

# ARNDCQEGHILKMFPSTYVWO
l = list()
for i in combinations_with_replacement('ARNDCQEGHILKMFPSTYVWO', 3):
    l.append(i)
    name = i[0] + i[1] + i[2]
    val = peptids[i[0]] + peptids[i[1]] + peptids[i[2]]
    c3_20[name] = val

сортируем их по возрастанию масс

In [662]:
sorted_omb = sorted(c3_20.items(), key=lambda item: item[1])

проверяем наличие в спектре

In [663]:
in_spectrum = dict()
for m, v in c3_20.items():
    if v in spectrum:
        in_spectrum[m] = v

print(in_spectrum)

{'ARY': 390, 'ADW': 372, 'AYW': 420, 'AWW': 443, 'RRD': 427, 'RRM': 443, 'RRP': 409, 'RDT': 372, 'RCI': 372, 'RCL': 372, 'RCM': 390, 'RES': 372, 'REW': 471, 'RHP': 390, 'RHV': 392, 'RFS': 390, 'RTY': 420, 'RTW': 443, 'RYW': 505, 'NNF': 375, 'NDY': 392, 'NEE': 372, 'NEF': 390, 'NIY': 390, 'NLY': 390, 'NMF': 392, 'NFO': 375, 'DQE': 372, 'DQF': 390, 'DEK': 372, 'DEM': 375, 'DIF': 375, 'DLF': 375, 'DKF': 390, 'DMY': 409, 'DFF': 409, 'DPY': 375, 'DYO': 392, 'CCW': 392, 'CMW': 420, 'CTW': 390, 'QEY': 420, 'QEW': 443, 'QIM': 372, 'QIW': 427, 'QLM': 372, 'QLW': 427, 'QMM': 390, 'QFP': 372, 'QFW': 461, 'QTY': 392, 'QYV': 390, 'QYW': 477, 'EEO': 372, 'EGW': 372, 'EKY': 420, 'EKW': 443, 'EMW': 446, 'EFV': 375, 'EFO': 390, 'GFW': 390, 'HHT': 375, 'HPW': 420, 'IKM': 372, 'IKW': 427, 'IMM': 375, 'IFW': 446, 'IYV': 375, 'IYO': 390, 'LKM': 372, 'LKW': 427, 'LMM': 375, 'LFW': 446, 'LYV': 375, 'LYO': 390, 'KMM': 390, 'KFP': 372, 'KFW': 461, 'KTY': 392, 'KYV': 390, 'KYW': 477, 'MMF': 409, 'MFP': 375, 'MF

In [664]:
print(max(spectrum))
print(max(spectrum)/min(peptids.values()))
print(max(spectrum)/max(peptids.values()))

1269
22.263157894736842
6.82258064516129


Точно какого размера наш белок мы не знаем, если ориентироваться на спектр, то он будет от 7 до 23 аминокислот

Попробуем сложить все наши самые частые аминокислоты и посмотреть на их массу. Может быть она будет примерно подходить

In [665]:
print(often)
s = 0
for a,n in often.items():    
    if n > 9:
        s += peptids[a]
print(s-113-128) # т.к. 113 и 128 встречаются дважды

{'P': 24, 'F': 23, 'Y': 22, 'E': 20, 'V': 20, 'N': 19, 'O': 19, 'I': 19, 'L': 19, 'D': 16, 'Q': 13, 'K': 13, 'M': 13, 'A': 9, 'T': 8, 'C': 6, 'R': 6, 'S': 5, 'G': 2, 'H': 1}
1350


Получилось что-то похожее на наш максимум спектра. Но что делать дальше понятнее не стало

Поищем среди 4 аминокислот

Напишем функцию, которая будет искать пептид на 1 аминокислоту больше чем было

In [666]:
def dora(variants:dict ,n: int)->dict:
    '''
    n - number of best scores. For examples: if you have peptids with scores 1,2,3,4,5 
    - you will get only peptides with score 4,5. That is n=2 highest scores  
    
    variants  - peptides with highest scores from last iteration of dora
    '''
    #ARNDCQEGHILKMFPSTYVWO
    allowed = 'ARNDCQEGHILKMFPSTYVWO'
    temp = dict()

    for pep, q in variants.items():
        for amin, e in peptids.items():
            if amin in allowed and q+e in spectrum:
                temp[pep+amin] = q+e               

    max_score = sorted(list(map(score, temp.keys())),key=lambda item: item[1])[::-1]

    scores = set()
    for tpl in max_score:
        scores.add(tpl[1])

    soccer = sorted(list(scores))[::-1]
    print('all scores', soccer)
    print('n - scores',soccer[:n])

    granted = dict()
    
    for l,m in max_score:
        if m in soccer[:n]:
            granted[l] = temp[l]
    print(granted)
    return granted

Получим возможные четверки

In [667]:
four = dora(in_spectrum,3)

all scores [7, 6, 5]
n - scores [7, 6, 5]
{'VWWF': 618, 'FOOM': 506, 'FWWV': 618, 'MFOO': 506, 'MFON': 506, 'MFPM': 506, 'MMFP': 506, 'LMMM': 506, 'IMMM': 506, 'EFOE': 519, 'EEOF': 519, 'NFOM': 506, 'NMFO': 506, 'NMFN': 506, 'NEFE': 519, 'NEEF': 519, 'NNFM': 506, 'RHPR': 546, 'RCMR': 546, 'RRPH': 546, 'RRMC': 546, 'VWWY': 634, 'VWWE': 600, 'SVWV': 471, 'SVWF': 519, 'FWWO': 633, 'FWWP': 616, 'FWWD': 634, 'FWWN': 633, 'FSWV': 519, 'FSWG': 477, 'KYWR': 633, 'KYVY': 553, 'KYVT': 491, 'KYVE': 519, 'KTYV': 491, 'KFPF': 519, 'LYOY': 553, 'LYOD': 505, 'IYOY': 553, 'IYOD': 505, 'GFWS': 477, 'GFWE': 519, 'EKYV': 519, 'EGWF': 519, 'QYWR': 633, 'QYVY': 553, 'QYVT': 491, 'QYVE': 519, 'QTYV': 491, 'QFPF': 519, 'QEYV': 519, 'CTWT': 491, 'CTWR': 546, 'DYOO': 506, 'DYOL': 505, 'DYOI': 505, 'DYON': 506, 'DPYM': 506, 'DMYP': 506, 'DKFE': 519, 'DKFD': 505, 'DEMM': 506, 'DEKF': 519, 'DQFE': 519, 'DQFD': 505, 'DQEF': 519, 'NLYY': 553, 'NLYD': 505, 'NIYY': 553, 'NIYD': 505, 'NDYO': 506, 'NDYL': 505, 'NDYI': 

Пройдём через пару итераций

In [668]:
five = dora(four,3)

all scores [12, 11, 10, 9, 8, 7]
n - scores [12, 11, 10]
{'QYVDE': 634, 'KYVDE': 634, 'ARYAR': 617, 'RFSRS': 633, 'DQFDE': 634, 'DKFDE': 634, 'EKYVD': 634, 'EEOFO': 633, 'EEOFN': 633, 'DEKVY': 634, 'QMMDE': 634, 'QMMED': 634, 'KMMDE': 634, 'KMMED': 634, 'ARYRA': 617, 'RYWQF': 780, 'RYWKF': 780, 'DQFDF': 652, 'DEKFD': 634, 'DEMMK': 634, 'DEMMQ': 634, 'DKFDF': 652, 'QEYVD': 634, 'QYVED': 634, 'KYVED': 634, 'RRMCR': 702, 'RRPHS': 633, 'RRPHR': 702, 'RCMRR': 702, 'RHPRS': 633, 'RHPRR': 702, 'EFOEO': 633, 'EFOEN': 633, 'FWWVY': 781, 'VWWFY': 781, 'ARYSR': 633, 'RHPSR': 633, 'RYWFK': 780, 'RYWFQ': 780, 'NEEVF': 618, 'NEEVE': 600, 'NEFRS': 633, 'NEFDF': 652, 'NEFDE': 634, 'NEFSR': 633, 'DQEVY': 634, 'DQEVF': 618, 'DQFRS': 633, 'DQFSR': 633, 'DKFRS': 633, 'DKFSR': 633, 'DFFPK': 634, 'DFFPQ': 634, 'QMMDM': 636, 'QFWRY': 780, 'EFORS': 633, 'EFOSR': 633, 'KMMDM': 636, 'KFWRY': 780, 'MFOIM': 636, 'MFOLM': 636, 'ARYAT': 562, 'ARYRS': 633, 'ARYTA': 562, 'AYWAA': 562, 'RFSRR': 702, 'RFSRA': 617, 'RFS

In [669]:
six = dora(five,3)
print(six)

all scores [18, 17, 16, 15, 14, 13, 12, 11, 10, 9]
n - scores [18, 17, 16]
{'DKFDQF': 780, 'DQFDKF': 780, 'NEFDFE': 781, 'ARYRAY': 780, 'RFSRAY': 780, 'DFFPQF': 781, 'DFFPKF': 781, 'DKFDFK': 780, 'DKFDFE': 781, 'DKFDFQ': 780, 'DQFDFK': 780, 'DQFDFE': 781, 'DQFDFQ': 780, 'MMFPQD': 749, 'MMFPKD': 749, 'EEOFDF': 781, 'FWWDFD': 896, 'KFPFDF': 781, 'QFPFDF': 781, 'DQEFDF': 781, 'MFOLML': 749, 'MFOLMI': 749, 'MFOIML': 749, 'MFOIMI': 749, 'DKFSRF': 780, 'DKFRSF': 780, 'DQFSRF': 780, 'DQFRSF': 780, 'DQEVYF': 781, 'NEFDEF': 781, 'DEKFDF': 781, 'KMMDED': 749, 'QMMDED': 749, 'DEKVYE': 763, 'DKFDEF': 781, 'DQFDEF': 781, 'RFSRSF': 780, 'ARYART': 718}
{'DKFDQF': 780, 'DQFDKF': 780, 'NEFDFE': 781, 'ARYRAY': 780, 'RFSRAY': 780, 'DFFPQF': 781, 'DFFPKF': 781, 'DKFDFK': 780, 'DKFDFE': 781, 'DKFDFQ': 780, 'DQFDFK': 780, 'DQFDFE': 781, 'DQFDFQ': 780, 'MMFPQD': 749, 'MMFPKD': 749, 'EEOFDF': 781, 'FWWDFD': 896, 'KFPFDF': 781, 'QFPFDF': 781, 'DQEFDF': 781, 'MFOLML': 749, 'MFOLMI': 749, 'MFOIML': 749, 'MFOIMI'

In [670]:
seven = dora(six,3)

all scores [25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13]
n - scores [25, 24, 23]
{'QFPFDFD': 896, 'KFPFDFD': 896, 'EEOFDFO': 895, 'EEOFDFN': 895, 'MFOIMIK': 877, 'MFOIMIQ': 877, 'MFOIMLK': 877, 'MFOIMLQ': 877, 'MFOLMIK': 877, 'MFOLMIQ': 877, 'MFOLMLK': 877, 'MFOLMLQ': 877, 'DQFDKFD': 895, 'DKFDQFD': 895, 'DQFDEFD': 896, 'DKFDEFD': 896, 'DQFDFQD': 895, 'DQFDFKD': 895, 'DKFDFQD': 895, 'DKFDFKD': 895, 'DFFPKFD': 896, 'DFFPQFD': 896}


In [671]:
eight = dora(seven,3)

all scores [37, 36, 35, 32, 30, 29, 28, 27, 26, 24, 22, 20]
n - scores [37, 36, 35]
{'DKFDQFDE': 1024, 'DQFDKFDE': 1024, 'DFFPQFDK': 1024, 'DFFPQFDQ': 1024, 'DFFPKFDK': 1024, 'DFFPKFDQ': 1024, 'DKFDFKDF': 1042, 'DKFDFKDE': 1024, 'DKFDFQDF': 1042, 'DKFDFQDE': 1024, 'DQFDFKDF': 1042, 'DQFDFKDE': 1024, 'DQFDFQDF': 1042, 'DQFDFQDE': 1024, 'DKFDQFDF': 1042, 'DQFDKFDF': 1042}


In [672]:
nine = dora(eight,3)

all scores [42, 41, 37, 34, 33, 32, 29, 27]
n - scores [42, 41, 37]
{'DQFDKFDFP': 1139, 'DKFDQFDFP': 1139, 'DQFDFQDED': 1139, 'DQFDFKDED': 1139, 'DKFDFQDED': 1139, 'DKFDFKDED': 1139, 'DQFDKFDED': 1139, 'DKFDQFDED': 1139, 'DQFDFQDFP': 1139, 'DQFDFKDFP': 1139, 'DKFDFQDFP': 1139, 'DKFDFKDFP': 1139, 'DFFPKFDQD': 1139, 'DFFPKFDKD': 1139, 'DFFPQFDQD': 1139, 'DFFPQFDKD': 1139}


In [673]:
ten = dora(nine,3)

all scores [52, 50, 49, 48, 38, 37, 36, 32]
n - scores [52, 50, 49]
{'DKFDFKDEDE': 1268, 'DKFDFQDEDE': 1268, 'DQFDFKDEDE': 1268, 'DQFDFQDEDE': 1268, 'DFFPQFDKDE': 1268, 'DFFPQFDQDE': 1268, 'DFFPKFDKDE': 1268, 'DFFPKFDQDE': 1268, 'DKFDQFDEDE': 1268, 'DQFDKFDEDE': 1268, 'DKFDFKDFPE': 1268, 'DKFDFQDFPE': 1268, 'DQFDFKDFPE': 1268, 'DQFDFQDFPE': 1268}


In [675]:
score('FPFFNEYVOL')

('FPFFNEYVOL', 26)

Я чего-то либо принципиально не понимаю, либо достичь желаемого тироцидина моим способом нельзя
FPFFNEYVOL (W)